import numpy as np
import matplotlib.pyplot as plt
import os
import neurokit2 as nk
from scipy.signal import butter, filtfilt, find_peaks, convolve
from scipy.spatial.distance import squareform, pdist
from skimage.transform import resize


In [2]:
def get_rri(qrs_raw_ind, fs):
    rri = np.diff(qrs_raw_ind) / fs  # Convert indices to time in seconds
    rri_milliseconds = rri * 1000
    return rri_milliseconds

In [ ]:
'''NeuraKit2 Based RR Interval Calculation'''

def neurakit_rri(record, fs):
    signals, info = nk.ecg_process(record, sampling_rate=fs, show = True)
    rpeaks = info["ECG_R_Peaks"]
    rpeak_amplitudes = record[rpeaks]
    rri = get_rri(rpeaks, fs)
    return rri, rpeak_amplitudes, rpeaks

save_dir = '/'
record = np.load(os.path.join(save_dir, 'p01_2.npy'))
fs = 128
rri, rpeak_amplitudes, rpeaks = neurakit_rri(record, fs)



In [ ]:
def normalize_to_gray_scale(matrix):
    min_val = np.min(matrix)
    max_val = np.max(matrix)
    normalized_matrix = (matrix - min_val) / (max_val - min_val + 1e-8)  # Avoid division by zero
    return (normalized_matrix*255).astype(np.uint8)

In [ ]:
def rp_plot(rri, delay, embedding_dim):
    N= len(rri)
    Nrp = N - (embedding_dim - 1) * delay
    embedded_rri = np.array([rri[i: i + embedding_dim * delay: delay] for i in range(Nrp)])
    distances = squareform(pdist(embedded_rri, metric='euclidean'))
    return distances

In [ ]:
img_dim = (224, 224)
print(rri)
distances = rp_plot(rri, delay= 3, embedding_dim=3)
rp_gray = normalize_to_gray_scale(distances)
rp_img = resize(rp_gray, img_dim, anti_aliasing=True, preserve_range=True).astype(np.uint8)

plt.imshow(rp_img, cmap='gray')
plt.axis('off')
plt.title("Recurrence Plot")
plt.show()